In [3]:
import gc
import json
import pandas as pd
import numpy as np

from glob import glob
from tqdm import tqdm

from utils import loadpkl, removeCorrelatedVariables, removeMissingVariables

%matplotlib inline

In [22]:
files = sorted(glob('../features/*.feather'))
df = pd.concat([pd.read_feather(f) for f in tqdm(files, mininterval=60)], axis=1)

# set card_id as index
df.set_index('card_id', inplace=True)
df.head()

  0%|          | 0/1079 [00:00<?, ?it/s]/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)
100%|██████████| 1079/1079 [00:01<00:00, 607.53it/s]


,AOV,AOV_ratio,CLV,CLV_month_diff,CLV_month_diff_ratio,CLV_ratio,Mothers_Day_2018,amount_month_ratio_max,amount_month_ratio_mean,amount_month_ratio_min,...,weekofyear_9,year_0,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,69.291900,0.011674,4.630503e+06,1.719647e+08,-0.000057,0.000091,49.492977,NaN,NaN,-50.80,...,0,0,0,0,0,0,0,0,1,0
C_ID_3d0044924f,88.296622,0.009137,1.071843e+07,1.005751e+07,0.000003,0.000003,56.833333,NaN,NaN,3.80,...,0,0,0,0,0,0,0,0,1,0
C_ID_d639edf6cd,46.450967,0.017520,8.440980e+04,NaN,NaN,0.000009,15.720930,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
C_ID_186d6a6901,64.539925,0.012546,3.779552e+05,NaN,NaN,0.000104,55.623377,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
C_ID_cdbd2c0db2,121.341267,0.006636,2.133293e+06,-9.454868e+07,0.000197,0.000486,61.856099,NaN,NaN,-72.19,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
# remove correlated variables
col_drop = removeCorrelatedVariables(df,0.9)
df.drop(col_drop, axis=1, inplace=True)

# remove missing variables
col_missing = removeMissingVariables(df,0.75)
df.drop(col_drop, axis=1, inplace=True)

Removing Correlated Variables...


In [5]:
feature_importance = pd.read_csv('../output/feature_importance_lgbm.csv')
feats_drop = feature_importance[feature_importance['importance']==0]['feature'].tolist()
feats_drop

['dayofweek_1',
 'hist_duration_unapproved_min',
 'hist_installments_unapproved_min',
 'hist_mer_active_months_lag12_skew_sum',
 'hist_mer_active_months_lag3_max_mean',
 'hist_mer_active_months_lag3_skew_sum',
 'hist_mer_active_months_lag3_var_mean',
 'hist_mer_active_months_lag3_var_sum',
 'hist_mer_active_months_lag6_skew_sum',
 'hist_mer_active_months_lag6_var_sum',
 'hist_mer_avg_active_months_skew_sum',
 'hist_mer_avg_numerical_skew_sum',
 'hist_mer_avg_purchases_lag12_skew_sum',
 'hist_mer_avg_purchases_lag3_skew_sum',
 'hist_mer_avg_purchases_lag6_skew_sum',
 'hist_mer_avg_purchases_skew_sum',
 'hist_mer_avg_sales_lag12_skew_sum',
 'hist_mer_avg_sales_lag3_skew_sum',
 'hist_mer_avg_sales_lag6_skew_sum',
 'hist_mer_avg_sales_skew_sum',
 'hist_mer_max_active_months_skew_sum',
 'hist_mer_max_purchases_skew_sum',
 'hist_mer_max_sales_skew_sum',
 'hist_mer_merchant_category_id_nunique_mean',
 'hist_mer_min_active_months_skew_sum',
 'hist_mer_min_purchases_skew_sum',
 'hist_mer_min_sa

In [13]:
# save feature name list
features_json = {}
features_json['features'] = [f for f in df.columns.tolist() if f not in feats_drop]
with open('../features/features_selected.json', 'w') as f:
    json.dump(features_json, f, indent=4)

In [5]:
df = loadpkl('../features/train_test.pkl')
hist = loadpkl('../features/historical_transactions.pkl')
new = loadpkl('../features/new_merchant_transactions.pkl')

In [6]:
# merge
df = pd.merge(df, hist, on='card_id', how='outer')
df = pd.merge(df, new, on='card_id', how='outer')

del hist, new
gc.collect()

NameError: name 'gc' is not defined

In [20]:
feat1= [
      "target",
      "outliers",
      "CLV_ratio",
      "amount_month_ratio_max",
      "amount_month_ratio_mean",
      "amount_month_ratio_min",
      "card_id_cnt_ratio",
      "card_id_cnt_total",
      "card_id_total",
      "category_1_mean",
      "days_feature1",
      "days_feature1_ratio",
      "days_feature2",
      "days_feature2_ratio",
      "days_feature3",
      "days_feature3_ratio",
      "duration_max",
      "duration_mean",
      "duration_min",
      "elapsed_time",
      "feature_1",
      "feature_2",
      "feature_3",
      "feature_max",
      "feature_mean",
      "feature_min",
      "feature_sum",
      "feature_var",
      "hist_Black_Friday_2017_mean",
      "hist_CLV",
      "hist_Children_day_2017_mean",
      "hist_Christmas_Day_2017_mean",
      "hist_Mothers_Day_2017_mean",
      "hist_Mothers_Day_2018_mean",
      "hist_Valentine_Day_2017_mean",
      "hist_amount_month_ratio_max",
      "hist_amount_month_ratio_mean",
      "hist_amount_month_ratio_min",
      "hist_amount_month_ratio_skew",
      "hist_amount_month_ratio_var",
      "hist_authorized_flag_mean",
      "hist_card_id_count",
      "hist_category_1_mean",
      "hist_category_2_mean",
      "hist_category_2_mean_mean",
      "hist_category_3_mean",
      "hist_category_3_mean_mean",
      "hist_day_max",
      "hist_day_mean",
      "hist_day_min",
      "hist_day_nunique",
      "hist_duration_max",
      "hist_duration_mean",
      "hist_duration_min",
      "hist_duration_skew",
      "hist_duration_var",
      "hist_fathers_day_2017_mean",
      "hist_first_buy",
      "hist_hour_max",
      "hist_hour_mean",
      "hist_hour_min",
      "hist_hour_nunique",
      "hist_installments_max",
      "hist_installments_mean",
      "hist_installments_skew",
      "hist_installments_sum",
      "hist_installments_var",
      "hist_is_holiday_mean",
      "hist_last_buy",
      "hist_merchant_category_id_nunique",
      "hist_merchant_id_nunique",
      "hist_month_diff_max",
      "hist_month_diff_mean",
      "hist_month_diff_skew",
      "hist_month_diff_var",
      "hist_month_lag_max",
      "hist_month_lag_mean",
      "hist_month_lag_min",
      "hist_month_lag_skew",
      "hist_month_lag_var",
      "hist_month_max",
      "hist_month_mean",
      "hist_month_min",
      "hist_month_nunique",
      "hist_price_max",
      "hist_price_mean",
      "hist_price_min",
      "hist_price_sum",
      "hist_price_var",
      "hist_purchase_amount_max",
      "hist_purchase_amount_mean",
      "hist_purchase_amount_min",
      "hist_purchase_amount_skew",
      "hist_purchase_amount_sum",
      "hist_purchase_amount_var",
      "hist_purchase_date_average",
      "hist_purchase_date_diff",
      "hist_purchase_date_uptomin",
      "hist_purchase_date_uptonow",
      "hist_subsector_id_nunique",
      "hist_weekday_mean",
      "hist_weekday_nunique",
      "hist_weekofyear_max",
      "hist_weekofyear_mean",
      "hist_weekofyear_min",
      "hist_weekofyear_nunique",
      "installments_max",
      "installments_mean",
      "installments_ratio",
      "installments_total",
      "month_diff_mean",
      "month_lag_mean",
      "new_Black_Friday_2017_mean",
      "new_CLV",
      "new_Children_day_2017_mean",
      "new_Christmas_Day_2017_mean",
      "new_Mothers_Day_2018_mean",
      "new_amount_month_ratio_max",
      "new_amount_month_ratio_mean",
      "new_amount_month_ratio_min",
      "new_amount_month_ratio_skew",
      "new_amount_month_ratio_var",
      "new_card_id_count",
      "new_category_1_mean",
      "new_category_2_mean",
      "new_category_2_mean_mean",
      "new_category_3_mean",
      "new_category_3_mean_mean",
      "new_day_max",
      "new_day_mean",
      "new_day_min",
      "new_day_nunique",
      "new_duration_max",
      "new_duration_mean",
      "new_duration_min",
      "new_duration_skew",
      "new_duration_var",
      "new_fathers_day_2017_mean",
      "new_first_buy",
      "new_hour_max",
      "new_hour_mean",
      "new_hour_min",
      "new_hour_nunique",
      "new_installments_max",
      "new_installments_mean",
      "new_installments_skew",
      "new_installments_sum",
      "new_installments_var",
      "new_last_buy",
      "new_merchant_category_id_nunique",
      "new_merchant_id_nunique",
      "new_month_diff_mean",
      "new_month_diff_skew",
      "new_month_diff_var",
      "new_month_lag_max",
      "new_month_lag_mean",
      "new_month_lag_min",
      "new_month_lag_skew",
      "new_month_lag_var",
      "new_month_max",
      "new_month_mean",
      "new_month_min",
      "new_price_max",
      "new_price_mean",
      "new_price_min",
      "new_price_skew",
      "new_price_var",
      "new_purchase_amount_max",
      "new_purchase_amount_mean",
      "new_purchase_amount_min",
      "new_purchase_amount_skew",
      "new_purchase_amount_sum",
      "new_purchase_amount_var",
      "new_purchase_date_average",
      "new_purchase_date_diff",
      "new_purchase_date_uptomin",
      "new_purchase_date_uptonow",
      "new_subsector_id_nunique",
      "new_weekday_max",
      "new_weekday_mean",
      "new_weekday_min",
      "new_weekend_mean",
      "new_weekofyear_max",
      "new_weekofyear_mean",
      "new_weekofyear_min",
      "new_weekofyear_nunique",
      "price_max",
      "price_mean",
      "price_total",
      "purchase_amount_max",
      "purchase_amount_mean",
      "purchase_amount_min",
      "purchase_amount_ratio",
      "purchase_amount_total",
      "quarter"
    ]

feat2 =[
      "feature_1",
      "feature_2",
      "feature_3",
      "first_active_month",
      "outliers",
      "target",
      "quarter",
      "elapsed_time",
      "days_feature1",
      "days_feature2",
      "days_feature3",
      "days_feature1_ratio",
      "days_feature2_ratio",
      "days_feature3_ratio",
      "month_year_8_2015",
      "feature_sum",
      "feature_mean",
      "feature_max",
      "feature_min",
      "feature_var",
      "hist_card_id_count",
      "hist_subsector_id_nunique",
      "hist_merchant_id_nunique",
      "hist_month_nunique",
      "hist_month_mean",
      "hist_month_min",
      "hist_month_max",
      "hist_hour_nunique",
      "hist_hour_mean",
      "hist_weekofyear_nunique",
      "hist_weekday_mean",
      "hist_day_nunique",
      "hist_day_mean",
      "hist_mer_category_1_sum_sum",
      "hist_mer_category_1_sum_mean",
      "hist_mer_category_4_sum_sum",
      "hist_mer_category_4_sum_mean",
      "hist_mer_category_2_-1_sum_sum",
      "hist_purchase_amount_sum",
      "hist_purchase_amount_max",
      "hist_purchase_amount_min",
      "hist_purchase_amount_mean",
      "hist_purchase_amount_var",
      "hist_purchase_amount_skew",
      "hist_purchase_amount_outlier_mean",
      "hist_installments_sum",
      "hist_installments_mean",
      "hist_installments_max",
      "hist_installments_var",
      "hist_installments_skew",
      "hist_purchase_date_min",
      "hist_month_lag_sum",
      "hist_month_lag_mean",
      "hist_month_lag_max",
      "hist_month_lag_min",
      "hist_month_lag_var",
      "hist_month_lag_skew",
      "hist_month_diff_sum",
      "hist_month_diff_mean",
      "hist_month_diff_max",
      "hist_month_diff_min",
      "hist_month_diff_var",
      "hist_month_diff_skew",
      "hist_authorized_flag_mean",
      "hist_category_1_mean",
      "hist_category_2_mean",
      "hist_category_3_mean",
      "hist_is_holiday_mean",
      "hist_price_max",
      "hist_price_min",
      "hist_price_mean",
      "hist_price_var",
      "hist_price_skew",
      "hist_Christmas_Day_2017_mean",
      "hist_Mothers_Day_2017_mean",
      "hist_fathers_day_2017_mean",
      "hist_Children_day_2017_mean",
      "hist_Black_Friday_2017_mean",
      "hist_Mothers_Day_2018_mean",
      "hist_duration_sum",
      "hist_duration_max",
      "hist_duration_min",
      "hist_duration_mean",
      "hist_duration_var",
      "hist_amount_month_ratio_sum",
      "hist_amount_month_ratio_max",
      "hist_amount_month_ratio_min",
      "hist_amount_month_ratio_mean",
      "hist_amount_month_ratio_var",
      "hist_amount_month_ratio_skew",
      "hist_category_2_mean_mean",
      "hist_category_3_mean_mean",
      "hist_purchase_date_average",
      "hist_purchase_date_diff",
      "hist_purchase_date_uptomin",
      "hist_purchase_date_uptonow",
      "hist_first_buy",
      "hist_last_buy",
      "new_card_id_count",
      "new_subsector_id_nunique",
      "new_merchant_id_nunique",
      "new_month_mean",
      "new_hour_mean",
      "new_hour_min",
      "new_hour_max",
      "new_weekofyear_nunique",
      "new_weekday_nunique",
      "new_weekday_mean",
      "new_weekday_min",
      "new_weekday_max",
      "new_day_mean",
      "new_day_min",
      "new_day_max",
      "new_mer_avg_sales_lag3_sum_sum",
      "new_purchase_amount_mean",
      "new_purchase_amount_skew",
      "new_purchase_amount_max",
      "new_purchase_amount_min",
      "new_purchase_amount_sum",
      "new_purchase_amount_var",
      "new_purchase_amount_outlier_sum",
      "new_purchase_amount_outlier_mean",
      "new_installments_sum",
      "new_installments_max",
      "new_installments_min",
      "new_installments_mean",
      "new_installments_var",
      "new_installments_skew",
      "new_purchase_date_max",
      "new_month_lag_max",
      "new_month_lag_min",
      "new_month_lag_mean",
      "new_month_lag_var",
      "new_month_lag_skew",
      "new_month_diff_sum",
      "new_month_diff_mean",
      "new_month_diff_max",
      "new_month_diff_min",
      "new_month_diff_var",
      "new_month_diff_skew",
      "new_weekend_mean",
      "new_category_1_sum",
      "new_category_1_mean",
      "new_price_min",
      "new_price_mean",
      "new_price_var",
      "new_Christmas_Day_2017_sum",
      "new_Christmas_Day_2017_mean",
      "new_Children_day_2017_sum",
      "new_Children_day_2017_mean",
      "new_Black_Friday_2017_sum",
      "new_Black_Friday_2017_mean",
      "new_Mothers_Day_2018_sum",
      "new_Mothers_Day_2018_mean",
      "new_duration_var",
      "new_duration_max",
      "new_duration_mean",
      "new_duration_min",
      "new_duration_skew",
      "new_amount_month_ratio_sum",
      "new_amount_month_ratio_max",
      "new_amount_month_ratio_mean",
      "new_amount_month_ratio_min",
      "new_amount_month_ratio_skew",
      "new_amount_month_ratio_var",
      "new_purchase_date_diff",
      "new_purchase_date_average",
      "new_purchase_date_uptomin",
      "new_purchase_date_uptonow",
      "new_first_buy",
      "new_last_buy",
      "card_id_cnt_ratio",
      "card_id_cnt_total",
      "amount_month_ratio_max",
      "amount_month_ratio_mean",
      "amount_month_ratio_min",
      "purchase_amount_ratio",
      "purchase_amount_max",
      "purchase_amount_mean",
      "purchase_amount_min",
      "purchase_amount_total",
      "month_diff_mean",
      "month_diff_max",
      "month_diff_min",
      "month_lag_mean",
      "month_lag_max",
      "month_lag_min",
      "duration_max",
      "duration_mean",
      "duration_min",
      "category_1_mean",
      "installments_max",
      "installments_mean",
      "installments_total",
      "installments_ratio",
      "price_max",
      "new_CLV",
      "hist_CLV",
      "new_AOV",
      "hist_AOV",
      "new_pred_CLV",
      "hist_pred_CLV",
      "new_CLV_month_diff_ratio",
      "hist_CLV_month_diff_ratio",
      "CLV_ratio",
      "AOV_ratio",
      "pred_CLV_ratio",
      "CLV_month_diff_ratio",
      "CLV",
      "AOV",
      "pred_CLV",
      "CLV_month_diff"
]

In [21]:
for f in feat1:
    if f not in feat2:
        print('"{}",'.format(f))

"card_id_total",
"hist_Valentine_Day_2017_mean",
"hist_day_max",
"hist_day_min",
"hist_duration_skew",
"hist_hour_max",
"hist_hour_min",
"hist_merchant_category_id_nunique",
"hist_price_sum",
"hist_weekday_nunique",
"hist_weekofyear_max",
"hist_weekofyear_mean",
"hist_weekofyear_min",
"new_category_2_mean",
"new_category_2_mean_mean",
"new_category_3_mean",
"new_category_3_mean_mean",
"new_day_nunique",
"new_fathers_day_2017_mean",
"new_hour_nunique",
"new_merchant_category_id_nunique",
"new_month_max",
"new_month_min",
"new_price_max",
"new_price_skew",
"new_weekofyear_max",
"new_weekofyear_mean",
"new_weekofyear_min",
"price_mean",
"price_total",


In [ ]:
df.hist_